In [1]:
!pip install flask flask-cors torch torchvision timm numpy opencv-python-headless pyngrok -q

import os
import torch
import timm
import numpy as np
import cv2
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf
from PIL import Image
from torchvision import transforms
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.7 MB/s eta 0:00:00


In [2]:
NGROK_AUTH_TOKEN = '2uPLcmFMRXK46DMBToujQlsoMcj_3fPsn9JzGNaFJfLrUPw1z'
MODEL_PATH = '/content/drive/MyDrive/model/best_model.pth'
drive.mount('/content/drive')
MODEL_ARCHITECTURE = "convnextv2_tiny.fcmae_ft_in1k"
NUM_CLASSES = 5
CLASS_NAMES = ['angry', 'happy', 'neutral', 'sad', 'surprise']

IM_SIZE = 224

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device.type}")

app = Flask(__name__)

CORS(app)

Mounted at /content/drive
Using device: cuda


In [3]:
model = None
try:
    model = timm.create_model(MODEL_ARCHITECTURE, pretrained=False, num_classes=NUM_CLASSES)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    model.to(device)
    model.eval()
    print("Emotion detection model loaded successfully.")
except Exception as e:
    print(f"Could not load model. Check the MODEL_PATH. Error: {e}")

preprocess_transform = transforms.Compose([
    transforms.Resize((IM_SIZE, IM_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

Emotion detection model loaded successfully.


In [4]:
@app.route("/")
def index():
    return "EmoTune PyTorch Backend API is running!"

@app.route("/predict", methods=["POST"])
def predict():

    if not model:
        return jsonify({"error": "Model is not loaded or failed to load."}), 500

    if 'image' not in request.files:
        return jsonify({"error": "No image file provided in the request."}), 400

    image_file = request.files['image']

    try:

        img = Image.open(image_file.stream).convert("RGB")

        input_tensor = preprocess_transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidence, predicted_index = torch.max(probabilities, 1)
            predicted_emotion = CLASS_NAMES[predicted_index.item()]

        return jsonify({
            "emotion": predicted_emotion,
            "confidence": confidence.item()
        })

    except Exception as e:
        print(f"Prediction failed: {str(e)}")
        return jsonify({"error": f"An error occurred during prediction: {str(e)}"}), 500

In [ ]:
if __name__ == "__main__":
    print("Starting Flask server with ngrok...")
    try:

        conf.get_default().auth_token = NGROK_AUTH_TOKEN

        public_url = ngrok.connect(5000)

        print(f"Backend is running and accessible at: {public_url}")

        app.run(port=5000, host='0.0.0.0')
    except Exception as e:

        print(f"Ngrok failed to start. Check your NGROK_AUTH_TOKEN. Error: {e}")

        print("Attempting to run Flask server locally on port 5000 without ngrok.")
        app.run(port=5000, host='0.0.0.0')

Starting Flask server with ngrok...
Backend is running and accessible at: NgrokTunnel: "https://e640-34-16-156-94.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:38:16] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:38:22] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:38:30] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:38:36] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:38:48] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:38:56] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:39:04] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:39:11] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Jun/2025 09:39:20] "POST /predic